# test-workbench

Testing deployment of RStudio IDE on Vertex AI Workbench

## Setup

### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
import os
# Get your Google Cloud project ID from gcloud
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null

try:
    PROJECT_ID = shell_output[0]
except IndexError:
    PROJECT_ID = None

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

### Enable APIS 

First, you must enable the following APIs for your project:

* Artifact Registry API 
* [Cloud Build API](https://console.cloud.google.com/flows/enableapi?apiid=cloudbuild.googleapis.com)

Learn more about [Enabling service](https://cloud.google.com/artifact-registry/docs/enable-service)

In [ ]:
! gcloud services enable artifactregistry.googleapis.com
! gcloud services enable cloudbuild.googleapis.com

## Build and push container image

### Create a private Docker repository

Your first step is to create your own Docker repository in Google Artifact Registry.

1.  Run the `gcloud artifacts repositories create` command to create a new Docker repository with your region with the description "docker repository".
    
2.  Run the `gcloud artifacts repositories list` command to verify that your repository was created.

In [ ]:
REGION = "us-central1" 
PRIVATE_REPO = "rstudio"

In [ ]:
! gcloud artifacts repositories create $PRIVATE_REPO --repository-format=docker \
    --location=$REGION \
    --description="Docker repository for RStudio on Vertex Workbench"

In [ ]:
! gcloud artifacts repositories list

In [ ]:
! gcloud auth configure-docker $REGION-docker.pkg.dev --quiet

### Build the Docker container

Next, you build the Docker container image on Cloud Build -- the serverless CI/CD platform.

_Note:_ Building the Docker container image will take ~15 minutes.

In [ ]:
IMAGE_NAME = "rstudio"
IMAGE_TAG = "latest" 
IMAGE_URI = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{PRIVATE_REPO}/{IMAGE_NAME}:{IMAGE_TAG}"
IMAGE_URI

In [ ]:
! gcloud builds submit ./Docker --region=$REGION --tag=$IMAGE_URI --timeout=1h

## Create Vertex AI Workbench Notebook

Start a new Notebook instance from the previously created container image. Make sure to choose a [custom container](https://cloud.google.com/ai-platform/notebooks/docs/custom-container). Pass in the gcr.io address you created earlier. Here's the url:

In [ ]:
REPO_URI = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{PRIVATE_REPO}/{IMAGE_NAME}"
REPO_URI

There are 2 ways to create a notebook instance 

1. Console UI 
2. `gcloud` command (below)

_Note:_ Creating the Workbench Notebook instance the first time will take ~5 minutes due to installation of dependencies

In [ ]:
# TODO - cleanup to be correct after testing complete
# ! gcloud notebooks instances create rstudio-instance --container-repository=$REPO_URI --container-tag=$IMAGE_TAG --machine-type=n1-standard-4 --location=us-central1-b

## Troubleshooting

Troubleshooting to get notebook working

### Download conatiner and run locally 

Download container and run locally to help debug in cloud shell

Run the following in a cloud shell instance: 


1. Authorize docker 

```sh
gcloud auth configure-docker us-central1-docker.pkg.dev --quiet
```

2. download the latest image to your local (cloud shell) directory

```sh
docker pull us-central1-docker.pkg.dev/demos-vertex-ai/rstudio/rstudio:latest
```

3. start container 

```sh
docker run --rm -p 8080:8787 0eb4550dc4ad
```

4. Generate url for RStudio 

```sh
echo https://8080-$WEB_HOST
```

5. click on the generated url to open rstudio